# Deep Belief Networks

In [99]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.datasets import mnist
from keras.layers import Dense, Layer
from keras.models import Sequential

# Load the MNIST dataset
(x_train, _), (x_test, _) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# Define the RBM layer
class RBM(Layer):
    def __init__(self, units):
        super(RBM, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='zeros',
                                 trainable=True)

    def call(self, inputs):
        h = tf.nn.sigmoid(tf.matmul(inputs, self.w) + self.b)
        return h

# Create the DBN Model
model = Sequential([
    RBM(256),  # First RBM layer
    RBM(64),    # Second RBM layer
    Dense(784, activation='sigmoid')  # Adding a dense layer to map back to original dimension
])

# Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, x_train, epochs=5)

# Visualize the weights of the first RBM layer
weights = model.layers[0].w.numpy().reshape(784, 256)

fig, axes = plt.subplots(16, 16, figsize=(10, 10))
for i, ax in enumerate(axes.flat):
    if i < 256:
        ax.imshow(weights[:, i].reshape(28, 28), cmap='gray')
        ax.axis('off')

plt.suptitle("Visualization of the First RBM Layer Weights in the DBN", fontsize=16)
plt.show()


Epoch 1/5


2023-12-24 01:28:29.101592: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: JIT compilation failed.


UnknownError: Graph execution error:

Detected at node sequential_6/rbm_10/Sigmoid defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/kali/rep/uj/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/kali/rep/uj/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1053, in launch_instance

  File "/home/kali/rep/uj/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/home/kali/rep/uj/.venv/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/home/kali/rep/uj/.venv/lib/python3.11/site-packages/ipykernel/kernelbase.py", line 516, in d